# Graph embedding

<hr>

# Index
## 0. train, validaion 데이터 준비
    - Train, Validation 데이터 전처리
    - GPU 설정
    - Graph 변수 준비
    - Arguments

## 1. GNN 모델 정의
    - Dataset & Loss function
    - LR-GCCF model
    
## 2. GNN 모델 학습
    - Train 정의
    - Validation 정의(with recall)
    - 학습 진행
<hr>

## 0. train, validaion 데이터 준비
    - Train, Validation 데이터 전처리
    - GPU 설정
    - Graph 변수 준비
    - Arguments

In [1]:
import pandas as pd
import numpy as np
import os
import time
from datetime import datetime, timedelta

In [2]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.preprocessing import LabelEncoder

In [2]:
# transactions data load
transactions = pd.read_csv('transactions_train.csv')
transactions['t_dat'] = pd.to_datetime(transactions['t_dat'], format='%Y-%m-%d')

first_date, last_date = min(transactions['t_dat']), max(transactions['t_dat'])
last_one_week = last_date + timedelta(weeks=-1, days=1)

train = transactions[transactions['t_dat']<last_one_week].copy()
validation = transactions[transactions['t_dat']>=last_one_week].copy()

### Train, Validation 데이터 전처리

In [3]:
# customer_id, article_id (in train data) mapping to integer
customer_train_list = train['customer_id'].unique()
article_train_list = train['article_id'].unique()
le_customer = LabelEncoder()
le_article = LabelEncoder()
le_customer.fit(customer_train_list)
le_article.fit(article_train_list)

implicit_log_df = train[['customer_id', 'article_id']].copy()
customer_id_label_np = le_customer.transform(implicit_log_df['customer_id'].to_numpy())
article_id_label_np = le_article.transform(implicit_log_df['article_id'].to_numpy())
implicit_log_df = pd.DataFrame(np.column_stack((customer_id_label_np, article_id_label_np)),\
                               columns = ['customer_id', 'article_id'])
customer_article_df = implicit_log_df.groupby('customer_id')['article_id'].apply(list)
article_customer_df = implicit_log_df.groupby('article_id')['customer_id'].apply(list)

customer_article_np = customer_article_df.reset_index().to_numpy()
article_customer_np = article_customer_df.reset_index().to_numpy()

LabelEncoder()

In [6]:
# validation data mapping to integer
customer_to_include = np.intersect1d(validation['customer_id'].unique(), train['customer_id'].unique(), \
                                     assume_unique=True)
article_to_include = np.intersect1d(validation['article_id'].unique(), train['article_id'].unique(), \
                                    assume_unique=True)

validation_df = validation[validation['customer_id'].isin(customer_to_include)]
validation_df = validation_df[validation_df['article_id'].isin(article_to_include)]

val_implicit_log_df = validation_df[['customer_id', 'article_id']].copy()
val_customer_id_label_np = le_customer.transform(val_implicit_log_df['customer_id'].to_numpy())
val_article_id_label_np = le_article.transform(val_implicit_log_df['article_id'].to_numpy())

val_implicit_log_df = pd.DataFrame(np.column_stack((val_customer_id_label_np, val_article_id_label_np)), \
                                   columns = ['customer_id', 'article_id'])
val_customer_article_df = val_implicit_log_df.groupby('customer_id')['article_id'].apply(list)
val_customer_article_np = val_customer_article_df.reset_index().to_numpy()

- 모델을 학습시키기 편하도록 train, validation 데이터의 customer_id와 article_id를 정수로 바꿔주는 작업이다.

### GPU 설정

In [7]:
# CUDA Setting
GPU_NUM = 3 # No special meaning. It's my GPU assigned from Lab.
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(device)

print('Device:', device)
print('Count of using GPUs:', torch.cuda.device_count())
print('Current cuda device:', torch.cuda.current_device())

Device: cuda:3
Count of using GPUs: 8
Current cuda device: 3


- GPU를 사용하여 학습할 예정이기 때문에 사용할 GPU를 지정해준다. 연구실 서버컴퓨터의 사용중이지 않은 GPU를 선택해주었다.

### Graph 변수 준비

In [ ]:
customer_reciprocal_degree = [1/len(arr) for arr in customer_article_np[:, 1]]
article_reciprocal_degree = [1/len(arr) for arr in article_customer_np[:, 1]]

- model을 training하는 데에 필요한 것으로, 각 node가 몇개의 다른 node들과 연결되어 있는지를 graph 이론에서 degree라고 표현하는데, 각 customer(or article)가 몇 개의 article(or customer)과 연결되어 있는지를 degree로 간주하여 이에 대한 역수를 저장하는 함수이다. 왜 역수를 저장하냐면, customer를 embedding할 때 연결된 article을 aggregating하여 벡터를 표현하는데, article 벡터들을 평균값으로 aggregate할 때 사용하기 위해 degree의 역수를 저장한다.(article은 반대로 진행) 해당 list와 바로 아래의 matrix를 만들고 활용하는 정확한 과정은 모델 설명 부분에서 자세히 다룰 것이다.

In [8]:
rows_customers = customer_article_np[:, 0]
cols_articles = customer_article_np[:, 1]
row_index = []
col_index = []
values_list = []
for i in range(len(customer_article_np)):
    row_index += [rows_customers[i]]*len(cols_articles[i])
    col_index += cols_articles[i]
    for col in cols_articles[i]:
        values_list.append(np.sqrt(customer_reciprocal_degree[rows_customers[i]]*article_reciprocal_degree[col]))
        
indices = torch.tensor([row_index, col_index])
customer_article_sparse_matrix = torch.sparse_coo_tensor(indices, values_list, dtype=torch.float32).to(device)
article_customer_sparse_matrix = torch.transpose(customer_article_sparse_matrix, 0, 1).to(device)

- 이 역시 model을 training하는 데에 필요한 matrix로, 위에서 언급한 user의 degree의 역수와 item의 degree의 역수를 곱한 후 square root를 씌워준 값을 element로 갖는 matrix를 생성한다.(${1 \over \sqrt{d_c \times d_a}}$; $d_c$:customer degree, $d_a$:article degree) 각 matrix는 \[customer$\times$aritcle\], \[article$\times$customer\]의 shape을 가진다. 이 역시 밑의 모델을 정의하는 부분에서 자세한 설명을 진행할 것이다.

### Arguments

In [9]:
args = {
    'factor': 64,
    'batch_size': 2048*300,
    'learning_rate': 0.001,
    'epochs': 100
}

- training을 할 때 사용할 각종 arguments들을 저장해둔 것이다.

> ```factor```: customer나 article을 embedding할 벡터의 dimension  
> ```batch_size```: batch 1개의 크기, GPU의 메모리만큼 허용되는 한 최대로 키움  
> ```learning_rate```: 학습률  
> ```epochs```: 전체 데이터를 학습할 횟수

## 1. GNN 모델 정의
    - Dataset & Loss function
    - LR-GCCF model

### Dataset & Loss function

In [10]:
class BPR_Dataset(torch.utils.data.Dataset):
    def __init__(self, customer_data_np, article_data_np, negative_sample_num=5, bought_article_num=0):
        super(BPR_Dataset, self).__init__()
        self.customer_data_np = customer_data_np
        self.article_data_np = article_data_np
        self.negative_sample_num = negative_sample_num
        self.bought_article_num = bought_article_num
        
    def negative_sampling(self):
        self.customer_pos_neg_dataset = []
        article_max = self.article_data_np.shape[0]
        article_array = np.arange(article_max)
        
        # for each customer
        for customer_articles in self.customer_data_np:
            positive_articles = customer_articles[1]
            
            # for each positive article
            for article_i in positive_articles:
                for _ in range(self.negative_sample_num):
                    neg_index = np.random.randint(article_max)
                    while neg_index in positive_articles:
                        neg_index = np.random.randint(article_max)
                    self.customer_pos_neg_dataset.append([customer_articles[0], article_i, \
                                                          article_array[neg_index]])
    
    def __len__(self):
        return self.negative_sample_num*self.bought_article_num

    def __getitem__(self, idx):
        customer = self.customer_pos_neg_dataset[idx][0]
        article_i = self.customer_pos_neg_dataset[idx][1]
        article_j = self.customer_pos_neg_dataset[idx][2] 
        return customer, article_i, article_j 
    
train_dataset = BPR_Dataset(customer_article_np, article_customer_np, negative_sample_num = 3, \
                            bought_article_num=implicit_log_df.shape[0])
train_loader = DataLoader(train_dataset, batch_size = args['batch_size'], shuffle = True, num_workers = 8)

In [12]:
def BPR_loss(customer_embedding, article_i_embedding, article_j_embedding):
    prediction_i = (customer_embedding*article_i_embedding).sum(dim=-1)
    prediction_j = (customer_embedding*article_j_embedding).sum(dim=-1)
    l2_reg = 0.01*(customer_embedding**2 + article_i_embedding**2 + article_j_embedding**2).sum(dim=-1).mean()
    
    loss = -((prediction_i - prediction_j)).sigmoid().log().mean() + l2_reg
    return loss

- ```BPR_Dataset```은 Pytorch의 [dataset tutorial](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html)을 참고하여 만들었다. 가장 핵심적인 부분은 ```negative_sampling```함수를 정의한 부분이다. 위의 ```BPR_loss```(Bayesian Personalized Raking)라는 loss function으로 학습을 진행할 예정인데, ```BPR loss```를 사용하기 위해서 ```negative sampling```이 필요해서 Dataset을 정의해준 것이다. 아래 loss의 수식을 보면 알 수 있지만, customer가 구매할 것 같은 article과의 embedding된 벡터 내적값과, 구매하지 않을 것 같은 article과의 embedding된 벡터 내적값 차이가 커지도록 학습이 진행된다. 
> $BPR\ loss = \Sigma_i^n log(\sigma(c_i\bullet a_i - c_i\bullet a_j)) + \gamma(c_i^2+a_i^2+a_j^2)$;  
> $c_i$: customer vector, $a_i$: sold article vector(positive item), $a_j$: arbitrary article vector(negative item)

    즉, 각 customer의 article 구매 이력이 담긴 training 데이터에 대해, 정말 구매했던 article(positive item)은 customer와의 내적값이 크도록, 구매이력이 없는 article(negative item)과는 내적값이 작도록 embedding된다. 해당 방식으로 학습을 진행하기 위해선 (customer, 구매이력이 있는 article, 구매이력이 없는 article)로 짝을 만들어야 하는데, 이를 위해 구매 이력이 없는 article인 negative item을 sampling 하려는 목적으로 ```negative_sampling```함수를 추가한 것이다. ```negative_sample_num```을 입력받아 positive item 하나당 몇개의 negative item을 sampling 할 지 결정하는데, 3을 사용했다.

### LR-GCCF model

In [11]:
class LRGCCF(nn.Module):
    def __init__(self, args, customer_article_matrix, article_customer_matrix, customer_deg, article_deg):
        super(LRGCCF, self).__init__()
        self.customer_article_matrix = customer_article_matrix
        self.article_customer_matrix = article_customer_matrix
        self.customer_deg = torch.unsqueeze(torch.cuda.FloatTensor(customer_deg), 1).expand(-1, args['factor'])
        self.article_deg = torch.unsqueeze(torch.cuda.FloatTensor(article_deg), 1).expand(-1, args['factor'])
        
        self.customer_embedding = nn.Embedding(self.customer_article_matrix.shape[0], args['factor'])
        self.article_embedding = nn.Embedding(self.customer_article_matrix.shape[1], args['factor'])
        nn.init.normal_(self.customer_embedding.weight, std=0.01)
        nn.init.normal_(self.article_embedding.weight, std=0.01)
        
    def forward(self, customer, article_i, article_j):
        customer_embedding = self.customer_embedding.weight
        article_embedding = self.article_embedding.weight
        
        layer_one_customer_embedding = torch.sparse.mm(self.customer_article_matrix, article_embedding)\
                                       + customer_embedding.mul(self.customer_deg)
        layer_one_article_embedding = torch.sparse.mm(self.article_customer_matrix, customer_embedding)\
                                      + article_embedding.mul(self.article_deg)
        layer_two_customer_embedding = torch.sparse.mm(self.customer_article_matrix, layer_one_article_embedding)\
                                       + layer_one_customer_embedding.mul(self.customer_deg)
        layer_two_article_embedding = torch.sparse.mm(self.article_customer_matrix, layer_one_customer_embedding)\
                                      + layer_one_article_embedding.mul(self.article_deg)
        
        total_customer_embedding = torch.cat((layer_one_customer_embedding, layer_two_customer_embedding), -1)
        total_article_embedding = torch.cat((layer_one_article_embedding, layer_two_article_embedding), -1)
        
        result_customer = F.embedding(customer, total_customer_embedding)
        result_article_i = F.embedding(article_i, total_article_embedding)
        result_article_j = F.embedding(article_j, total_article_embedding)
        
        return result_customer, result_article_i, result_article_j

model = LRGCCF(args, customer_article_sparse_matrix, article_customer_sparse_matrix, customer_reciprocal_degree,\
               article_reciprocal_degree)
model.to(device)

LRGCCF(
  (customer_embedding): Embedding(1356709, 64)
  (article_embedding): Embedding(103880, 64)
)

- ```nn.Embedding()```, ```nn.init.normal_()```: model class를 정의하고 init부분에서 모든 customer와 article에 대해 64-dimension으로 embedding matrix를 선언해준다. 이후 표준편차(**st**andard **d**eviation) = 0.01의 normal distribution으로 initializing을 진행한다.
- init의 다른 부분으로 ```customer_article_matrix```와 ```article_customer_matrix```가 존재하는데, 이는 각각 \[row: customer, column: article\], \[row: article, column: customer\]인 sparse matrix이다. 해당 matrix를 만든 이유를 설명하려면  graph neural network의 학습 방법을 설명해야 한다. 이는 아래에서 그림과 함께 설명하겠다.
    
    <table>
    <tr>
    <td><img src="./bipartite_graph.png"><figcaption class="figure-caption text-center">Bipartite graph</figcaption></td>
    <td><img src="./degree_matrix.png"><figcaption class="figure-caption text-center">(reciprocal) Degree matrix</figcaption></td>
    </tr>
    </table>
    
- 우리가 사용했던 customer가 article을 구매했던 데이터들을 왼쪽 그림과 같이 bipartite(노드의 종류가 이분화된) graph로 표현할 수 있다. 각 customer와 article들을 node로 표현하고, 구매기록이 있다면 edge로 연결을 해줌으로써 표현해주었다.  
- Graph 자료구조에서 node에 연결된 edge의 갯수를 degree라고 표현하는데, 이 degree의 역수(이하 reciprocal degree)를 곱해준 값을 \[row: customer, column: article\]인 matrix의 element로 사용하면 오른쪽 그림과 같이 표현할 수 있다. 각 node에 할당된 가중치를 1로 통일하였을 때(고객이 article에 대해 얼마나 만족하였는지를 알 수 있는 별점같은 정보 없이 단순히 구매를 했다 안 했다 정도만 알 수 있기에 1로 통일), 각 node별로 구매한 article의 수가 다르므로 이를 반영하는 normalize를 하기 위해 reciprocal degree를 사용한다. 예를 들면 $c_1$은 3개의 article을 구매했고 $a_2$는 2명의 고객에게 판매되었으므로, ($c_1$, $a_2$) element엔 ${1\over3}\times{1\over2}$이 들어가있다.

- 위에서 기존 구매이력들을 바탕으로 bipartite graph로 모델링하고, 이를 어떻게 matrix로 나타내는지 알아봤다. 이제는 이를 중심으로 ```forward```에서 벌어지는 일들을 이해해보자. ```forward```에선 customer와 article을 embedding하는 layer가 각각 존재하는데, 그중에서 customer를 embedding하는 방법을 살펴보겠다. article을 embedding 하는 방법도 customer를 embedding 하는 방법과 같아서 정반대의 변수들로 같은 연산을 해주면 된다. ```forward```는 아래와 같이 세 단계의 연산을 수행한다.

    <table>
    <tr>
    <td><img src="./mm_left.png"><figcaption class="figure-caption text-center">matrix mulplication</figcaption></td>
    <td><img src="./mm_right.png"><figcaption class="figure-caption text-center">output matrix</figcaption></td>
    </tr>
    </table>
    
> 1) ```customer_article_matrix```$\times$ ```article_embedding```
    <pre> 왼쪽 사진과 같이 matrix multiplication을 하게 되면, 왼쪽 matrix의 row 벡터와(하늘색 박스) 오른쪽 matrix의 column벡터(빨간색 박스)의 내적값으로 오른쪽 사진과 같이 output matrix의 원소들이 표현된다. Output matrix의 (customer1, dim-0)원소를 보면 graph neural network이 어떻게 reciprocal degree를 이용하여 graph 데이터의 관계성을 포착하는지 이해할 수 있다. customer1의 dim-0는 customer1이 구매한 적이 있는 article들의 dim-0 element들을 aggregate하여 표현한다.(구매 이력이 없다면 matrix의 해당 원소들이 0으로 채워져 있어서 aggregate되지 않는다) customer1이 3개의 article과 연결되어 있으므로 3개의 article에 대한 평균을 취하기 위해 가장 앞에 1/3이 곱해져 있다. article1의 경우 1명, article 2의 경우 2명, article3의 경우 1명에게 판매되었으므로 그에 해당되는 reciprocal degree가 각 article의 dim-0원소에 곱해져있다.
 이를 종합해보자면 customer 벡터의 각 원소들은, 연결된(구매했던) article들의 해당되는 dim의 원소들인 reciprocal degree들을 평균내어 반영한다. 이때 구매이력이 없는 경우 reciprocal degree가 0이므로 결과 matrix에 반영이 되지 않는다. 이러한 일련의 과정들을 graph neural network 분야에서 'message passing'이라고 하는데, 참으로 직관적인 네이밍이라고 생각하는게, article에서 각 dim의 원소들을 normalize해서 연결된 customer에 'message'를 'passing'하듯이 전달한 것이다.
 위와 같은 message passing을 반복하면서 좀 더 고차원의 관계성을 포착할 수 있다. 예를 들면 customer 1은 article 1, 2, 3을 구매했다. 이때 customer2 역시 article 2를 구매했다. 향후 customer 2가 article k를 구매하게 된다면, article k를 customer 1에게도 추천해줄 수 있는 것이다. 위와 같은 형태의 graph에서 node간(customer와 직접 연결된 article)의 거리를 1-hop이라는 단위로 정의하는데, 이에 따르면 customer 1과 customer 2는 2-hop의 거리만큼 떨어져있다. 우리가 사용한 모델에서 k개의 layer를 사용한다면 k-hop 떨어진 node까지 message passing을 할 수 있다.</pre>  
2) ```customer_embedding```$\times {1 \over d_c}$(customer의 degree 역수)  
    <pre>1)에서는 주변 node들로부터 message를 passing받았다면, 2)에서는 기존 node의 message를 가져오는 것을 말한다. 구체적인 용어로는 self-loop이라고 하는데, 당연하게도 이런 self-loop이 있어야 node가 기존 정보를 기억하고 있을 것이다.
</pre>  
3) 1) + 2)  
    <pre>1)과 2)를 더해줌으로써, 2)에서의 기존 정보에 1)에서의 message passing으로 인접한 node들로부터 얻은 정보를 더해 업데이트를 진행한다.</pre> 

- 위와 같이 Graph Convolution을 이용한 Collaborative Filtering을 진행하여 모델이름이 LR-GCCF이다. 모델의 full name은 Linear Residual Graph Convolutional Collaborative Filtering인데, 이제 모델 이름에 Linear와 Residual이 왜 붙었는지 설명할 예정이다.
- ```Linear```: 여타의 neural network 방법론에서 한개의 layer를 통과할 때 보통 non-linaer activation function을 사용한다. Graph neural network도 역시 non-linaer activation function을 똑같이 사용하는 모델이 많은데, [Simplifying Graph Convolutional Networks](https://arxiv.org/pdf/1902.07153.pdf)\[Wu et al. ICML 2019\]에 따르면 GCN에선 non-linear activation function을 사용하지 않아도 성능이 크게 감소되지 않으면서도 훨씬 빠르게 학습할 수 있다고 한다. 위의 GCN 연산과정을 돌이켜 보면 이미 주변 node들의 정보들을 충분히 aggregate 하기도 하고, 연결된 node들을 그 자체로 유사성을 갖기 때문에 non-linear activation function 없이도 표현력이 그다지 제한되지 않는다고 한다. 그래서 Linear 하다는 것은 non-linear activation function을 사용하지 않는다는 의미이다.
- ```Residual```: 이건 computer vision 분야에서 사용되는 ResNet에서 차용한 아이디어이다. ```forward()```에서 보면 2개의 layer를 통과하여 ```total_customer_embedding```변수에서 이전에 통과했던 layer를 합쳐서 사용하는 것을 볼 수 있다. 어떤 layer가 optimal layer인지 deep하게 쌓아나가다보면 모를 수 있고, 특정 layer에서 특정 부분은 optimal이만 다른 부분은 optimal이 아닐 수도 있어 해당 방식으로 모두 사용해주게 되면 가장 마지막 layer가 아니더라도 최종적인 output과 직접적으로 연결된다고 볼 수 있는 skip-connection을 만들어주어 성능을 더 끌어올릴 수 있다.

## 2. GNN 모델 학습
    - Train 정의
    - Validation 정의(with recall)
    - 학습 진행

### Train 정의

In [15]:
def train(train_loader, model, optimizer):
    train_loader.dataset.negative_sampling()   
    
    for customer, article_i, article_j in train_loader:
        customer = customer.cuda()
        article_i = article_i.cuda()
        article_j = article_j.cuda()

        optimizer.zero_grad()
        result_customer, result_article_i, result_article_j = model(customer, article_i, article_j)
        loss = BPR_loss(result_customer, result_article_i, result_article_j)
        loss.backward()
        optimizer.step()
        
    return loss

- train에서는 먼저 위의 ```BPR_Dataset```에서 언급한 negative sampling을 진행해준다. 그리고 (customer, 실구매 article(positive item), 미구매 article(negative item))을 load한 후, for loop를 통해 학습을 진행한다. 아래는 for loop 내부 동작을 설명한 것이다.

> 1. GPU로 data load  
> 2. optimizer 초기화(밑에 나와있지만 Adam을 optimizer로 사용)  
> 3. model에 data를 통과시켜 embedding된 벡터 얻어내기  
> 4. embedding된 벡터를 BPR_loss로 계산
> 5. backward propagation을 통해 갱신할 parameter들에 loss에 대한 편미분 값 전달
> 6. optimizer를 통해 모델 train


### Validation 정의(with recall)

In [28]:
def validation(model, val_customer_article_np):
    customer_embedding_result = model.customer_embedding.weight.clone().cpu().detach().numpy()
    article_embedding_result = model.article_embedding.weight.clone().cpu().detach().numpy()
    
    matmul_cus_art = np.matmul(customer_embedding_result[val_customer_article_np[:, 0].astype(int)],\
                               article_embedding_result.T)
    top_hundred_articles = np.argpartition(matmul_cus_art, -100)[:, -100:]
    recall_list = []
    for top_hundred_article, val_articles in zip(top_hundred_articles, val_customer_article_np[:, 1]):
        recall_list.append(len(np.intersect1d(val_articles, top_hundred_article))/len(val_articles))

    return np.array(recall_list).mean()

- validation에선 ```transactions_train.csv```의 마지막 일주일간의 log 데이터인 validation 데이터를 이용하여 검증을 한다. Graph embedding을 이용하여 결국 하고자 하는 것은 각 고객에게 알맞은 article candidate을 구성하는것이므로, validaion 데이터에 나타난 고객들의 article 구매데이터를 얼마나 잘 맞추는지 평가하기 위해 recall을 사용할 것이다. validaion에 나타난 고객 데이터에 대해 recall을 각각 구하고, 이들을 모두 더해 평균을 내는 방식으로 평가를 진행할 것이다.

### 학습 진행

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=args['learning_rate'])

for epoch in range(args['epochs']):
    loss = train(train_loader, model, optimizer)
    print(epoch+1, '-th epoch, train mean loss:', loss)
    if (epoch+1) % 5 == 0:
        recall_at_hundred = validation(model, val_customer_article_np)
        print('validation, recall@100:', recall_at_hundred)
        PATH_model = os.path.join(os.getcwd(), 'graph_training_result', str(epoch+1))
        torch.save(model.state_dict(), PATH_model)

- Optimizer로 Adam을 선언하고 위에서 ```args(arguments)```에 정의했던 대로 100번의 epoch만큼 training을 진행할 것이다. validaion이 시간이 오래걸려서 매 5 epoch마다 validaion을 진행하여 recall 평균값을 출력하고, validaion을 진행할 때마다 모델을 저장할 것이다. 향후 가장 높은 recall 값이 나온 epoch의 모델을 사용하여 candidate을 구성할 것이다.

- 모델 학습에 상당히 시간이 오래 걸려서 위의 코드를 모두 복사하여 별도의 python 파일을 만들어 학습을 진행시켰다. 학습결과 30 epoch에서 가장 높은 recall 값을 보여주었기에 candidate 생성은 30 epoch의 학습을 진행했을 때의 embedding된 벡터로 진행할 것이다.